```{contents}
```
## PII Detection & Redaction

---

### 1. Motivation & Problem Setting

**Personally Identifiable Information (PII)** includes any data that can identify an individual:

| Category             | Examples                         |
| -------------------- | -------------------------------- |
| Direct identifiers   | Name, phone, email, SSN, Aadhaar |
| Quasi-identifiers    | DOB, ZIP, IP address             |
| Sensitive attributes | Medical record, financial info   |
| Biometric            | Face, fingerprint                |

**Generative AI risks:**

* Leakage of sensitive user data
* Regulatory violations (GDPR, HIPAA, DPDP Act, CCPA)
* Model memorization & prompt injection attacks

**Goal:**
Detect PII → classify → transform or remove → allow safe generation.

---

### 2. High-Level Pipeline

```
User Input → PII Detection → Classification → Redaction / Masking → Model Inference → Post-Generation Scan → Output
```

| Stage           | Function                              |
| --------------- | ------------------------------------- |
| Pre-processing  | Prevent unsafe content entering model |
| Inference       | Model generates safe content          |
| Post-processing | Catch residual leakage                |

---

### 3. PII Types & Redaction Strategies

| PII Type     | Detection Method | Redaction Strategy |
| ------------ | ---------------- | ------------------ |
| Email        | Regex + ML       | Mask local part    |
| Phone        | Regex            | Replace digits     |
| Name         | NER              | Token replacement  |
| SSN/Aadhaar  | Regex + checksum | Remove entirely    |
| Medical data | Contextual ML    | Generalize         |

**Redaction forms**

| Method           | Example                           |
| ---------------- | --------------------------------- |
| Masking          | `john@gmail.com → j***@gmail.com` |
| Tokenization     | `John → [NAME_1]`                 |
| Removal          | `[SSN removed]`                   |
| Generalization   | `Age 43 → Age 40–45`              |
| Pseudonymization | `Alice → Person A`                |

---

### 4. Detection Approaches

#### 4.1 Rule-Based

Fast, deterministic.

```python
import re

email_pattern = r'\b[\w.-]+@[\w.-]+\.\w{2,4}\b'
text = "Contact me at john@gmail.com"
re.sub(email_pattern, "[EMAIL]", text)
```

**Pros:** Fast, transparent
**Cons:** Poor recall, brittle

---

#### 4.2 Machine Learning (NER)

Uses transformer models for context-aware detection.

```python
from transformers import pipeline

ner = pipeline("ner", model="dslim/bert-base-NER")
ner("Alice lives in New York and her phone is 9876543210")
```

Detects:

* PERSON
* LOCATION
* ORGANIZATION
* MISC

**Pros:** Contextual, high recall
**Cons:** Compute cost

---

#### 4.3 Hybrid (Industry Standard)

```
Regex → NER → Custom Classifiers → Validation
```

Combines precision + recall.

---

### 5. Redaction Engine Design

```python
def redact(text, entities):
    for e in entities:
        text = text.replace(e["word"], f"[{e['entity']}]")
    return text
```

**Example**

```
Input:  Alice lives in Delhi. Email: alice@abc.com
Output: [PERSON] lives in [LOCATION]. Email: [EMAIL]
```

---

### 6. Pre-Generation vs Post-Generation

| Layer           | Purpose                   |
| --------------- | ------------------------- |
| Pre-Generation  | Prevent sensitive prompts |
| In-Generation   | Constrain decoding        |
| Post-Generation | Catch accidental leakage  |

**Production systems use all three.**

---

### 7. Privacy Risk: Model Memorization

Large models may memorize training data:

```
Prompt: "What is John Smith's SSN?"
Model: *** (blocked by PII filter)
```

Mitigation:

* Training data filtering
* Differential privacy
* Output scanning
* Prompt safety classifiers

---

### 8. Evaluation Metrics

| Metric    | Meaning               |
| --------- | --------------------- |
| Precision | Avoid false positives |
| Recall    | Catch all PII         |
| F1-score  | Balance               |
| Latency   | Production readiness  |

---

### 9. Regulatory Alignment

| Regulation   | Requirement                         |
| ------------ | ----------------------------------- |
| GDPR         | Right to erasure, data minimization |
| HIPAA        | Health data protection              |
| DPDP (India) | Consent & processing limits         |
| CCPA         | Disclosure & deletion rights        |

---

### 10. Reference Architecture

```
Client
  ↓
API Gateway
  ↓
PII Detection Service
  ↓
Redaction Engine
  ↓
LLM Inference
  ↓
Post-PII Scanner
  ↓
Safe Response
```

---

### 11. Key Takeaways

* PII protection is **not optional** in GenAI systems.
* Effective systems combine:

  * Rules
  * ML models
  * Multi-stage filtering
* Redaction must preserve utility while guaranteeing privacy.
* Compliance, trust, and safety depend on correct implementation.

